## Demo on the DREimac sphere data

Here we implement our idea for a function H that can be used to visualize our space from higher dimensions. We work with the following well behaved data set from DREimac.

In [1]:
import numpy as np
import plotly.graph_objects as go
from ripser import ripser
from persim import plot_diagrams

import matplotlib.pyplot as plt
%matplotlib inline

import math
from definitions import angle_from_side

from dreimac import GeometryExamples

from numpy import linalg, matmul
from scipy import special


In [2]:
#Load data
sphere = GeometryExamples.sphere(10000)

In [21]:
#Display data
fig = go.Figure(data=[go.Scatter3d(
    x=sphere[:,0], y=sphere[:,1], z=sphere[:,2], 
    mode ='markers', 
    marker=dict(size = 3, opacity = 1, color='grey')
)])

fig.update_layout(autosize=False, width=700, height=700)  

fig.show()

In [ ]:
#Using Ripser, we will find the persistence diagrams
#Here I am referecing code from Demo 6 in Day 5 DREimac

n_land = 900
res = ripser(sphere, n_perm = n_land, maxdim=2, coeff =13, thresh = 1.8)
dgms = res['dgms']

In [ ]:
fig = plt.figure(figsize = (4,4))
plot_diagrams(dgms)
fig.savefig('sphere-dreimac-persistence')

sphericalcoords is used to convert our data into spherical coordinates ordered $(r, \theta, \phi)$ to implement the spherical harmonics

In [25]:
sphericalcoords = []
for i in sphere:
    r = math.sqrt((i[0])**2 + (i[1])**2 + (i[2])**2)
    w = math.sqrt((-i[0])**2 + (i[1])**2)
    theta = math.acos((i[2])/r)
    phi = angle_from_side(i[0], i[1], w)
    sphericalcoords.append([r, theta, phi])

sphericalcoords = np.array(sphericalcoords)

Setting our parameters, $D$ is fixed to be the number of data points in our sample. $N$ is a parameter we choose.
In this implementation, we use the function $linalg.pinv$ which computes the Moore-Penrose pseudoinverse. This function uses the method called singular value decomposition, SVD, to compute this inverse. $N = 85$, is the last time the SVD successfully converges.

The matrix $A$ consists of the evaluated spherical harmonics at every point in the data. The shape is a $10,000 \times 3$ matrix, where 10,000 was the number of data points we uploaded. Below are two methods for sampling a portion of spherical harmonics. The commented block does not properly sample the spherical harmonics in its top down triangle model.

In [34]:
# D = sphere.shape[0]
# N = 85
# A = []

# for i in range(D):
#     for j in range(N):
#         A.append(special.sph_harm(j+1, N, sphericalcoords[i][1], sphericalcoords[i][2]))
# A = np.array(A)
# A = A.reshape(D,N)

In [ ]:
D = penta1.shape[0]
N = 100
mat = []

for row in range(N):
    for col in range(row+1):
        if col == 0 :
            mat.append((row, col))
        else:
            mat.append((row, col))
            mat.append((row, -1*col))

func = mat[:N]

A = []

for i in range(D):
    for ind in func:
        A.append(special.sph_harm(ind[1], ind[0], sphericalcoords[i][1], sphericalcoords[i][2]))

A = np.array(A)
A = A.reshape(D,N)

In [35]:
#Compute inverse
A_inv = linalg.pinv(A)
u = matmul(A_inv, sphericalcoords)

The error function below is the code to implement $\lVert b - A\cdot u \rVert^2$, where $b$ is the spherical coordinates data set and u is our estimated coefficients $\hat{h}_d$

Then we determine the average of the error per data point, which is the displayed value below:

NOTE: The commented block only handles the sphere and never projects into $\mathbb{C}^5$ as we would desire.

In [36]:
# error = (linalg.norm(sphericalcoords - matmul(A, u)))
# error = error/sphere.shape[0]
# print(error)

error = (linalg.norm(sphere - matmul(A, u)))
error = error/sphere.shape[0]
print(error)

0.011060102981002974


In [ ]:
# error_param = []

# for param in range(86):
#     matrix = []
#     for i in range(D):
#         for j in range(param):
#             matrix.append(special.sph_harm(j+1, param, sphericalcoords[i][1], sphericalcoords[i][2]))
#     matrix = np.array(matrix)
#     matrix = matrix.reshape(D,param)
#     matrix_inv = linalg.pinv(matrix)
#     u = matmul(matrix_inv, sphericalcoords)
#     error_prime = (linalg.norm(sphericalcoords - matmul(matrix, u)))
#     error_prime = error_prime/sphere.shape[0]
#     error_param.append(error_prime)
    
    
start_time = time.time()
error_param = []

for param in range(100):
    mat = []
    for row in range(param):
        for col in range(row+1):
            if col == 0 :
                mat.append((row, col))
            else:
                mat.append((row, col))
                mat.append((row, -1*col))
    func = mat[:param]
    
    A = []
    for i in range(D):
        for ind in func:
            A.append(special.sph_harm(ind[1], ind[0], sphericalcoords[i][1], sphericalcoords[i][2]))
    A = np.array(A)
    A = A.reshape(D,param)
    A_inv = linalg.pinv(A)
    u = matmul(A_inv, sphericalcoords)
    error = (linalg.norm(sphericalcoords - matmul(A, u)))
    error = error/sphere.shape[0]
    error_param.append(error)

end_time = time.time()
print(end_time - start_time)

In [ ]:
plt.scatter(range(86), error_param)

In [ ]:
plt.plot(range(86), error_param)

# Another Method for finding the Local Left Inverse